<a href="https://colab.research.google.com/github/Anurag0207/Anurag0207/blob/main/emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
for dirname, _, filenames in os.walk('/content/your_dataset_directory'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
import keras
from tensorflow.keras.optimizers import Adamax
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Model,load_model
from keras.models import Model
from keras.layers import Input,ELU ,Dense ,Conv2D ,Add ,Activation , Flatten, Dropout , GlobalAveragePooling2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, AveragePooling2D, Subtract, Concatenate, ZeroPadding2D,Conv2DTranspose,SeparableConv2D
import tensorflow as tf
from keras import regularizers

In [4]:
!pip install gdown
file_id = "1Bkr0Ii8JKUOHGFq71NkzGuqJpoG1mYrD"
output_file = "/content/dataset.zip"

import gdown
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file, quiet=False)

import zipfile
with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall("/content")



Downloading...
From: https://drive.google.com/uc?id=1Bkr0Ii8JKUOHGFq71NkzGuqJpoG1mYrD
To: /content/dataset.zip
100%|██████████| 101M/101M [00:02<00:00, 42.0MB/s] 


In [5]:
def generate_dataset():
    df = pd.read_csv("/content/fer2013/fer2013/fer2013.csv")

    train_samples = df[df['Usage'] == "Training"]
    validation_samples = df[df["Usage"] == "PublicTest"]
    test_samples = df[df["Usage"] == "PrivateTest"]

    y_train = keras.utils.to_categorical(train_samples.emotion, num_classes=7)
    y_valid = keras.utils.to_categorical(validation_samples.emotion, num_classes=7)
    y_test = keras.utils.to_categorical(test_samples.emotion, num_classes=7)

    X_train = np.array([np.fromstring(image, np.uint8, sep=" ").reshape((48, 48)) for image in train_samples.pixels])
    X_valid = np.array([np.fromstring(image, np.uint8, sep=" ").reshape((48, 48)) for image in validation_samples.pixels])
    X_test = np.array([np.fromstring(image, np.uint8, sep=" ").reshape((48, 48)) for image in test_samples.pixels])

    return X_train, y_train, X_valid, y_valid, X_test, y_test

x_train, y_train, x_valid, y_valid, x_test, y_test = generate_dataset()


In [6]:
first_image=x_train[0]

In [7]:
def generate_model(lr=0.001):
        myinput=Input(shape=(48,48,1))
        block1_layer11=SeparableConv2D(32,(3,3), padding="valid",data_format='channels_last',kernel_regularizer=regularizers.l2(0.00001),name='block1_sep_conv1')(myinput)
        block1_layer1=Activation('relu',name='block1_relu1')(block1_layer11)
        block1_layer1=BatchNormalization(name='block1_batch1')(block1_layer1)
        block1_layer1=Dropout(0.1,name='block1_drop1')(block1_layer1)
        block1_layer2=SeparableConv2D(32,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block1_sep_conv2')(block1_layer1)
        block1_layer2=Activation('relu',name='block1_relu2')(block1_layer2)
        block1_layer2=BatchNormalization(name='block1_batch2')(block1_layer2)
        block1_layer2=Dropout(0.1,name='block1_drop2')(block1_layer2)
        block1_layer3=SeparableConv2D(32,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block1_sep_conv3')(block1_layer2)
        block1_add1=Add(name='block1_add')([block1_layer3,block1_layer11])
        block1_layer3=Activation('relu',name='block1_relu')(block1_add1)
        block1_layer3=BatchNormalization(name='block1_batch3')(block1_layer3)
        block1_layer3=ZeroPadding2D((3,3),name='block1_zeroo')(block1_layer3)
        block1_layer3=AveragePooling2D(name='block1_avg')(block1_layer3)
        block1_layer3=Dropout(0.1,name='block1_drop3')(block1_layer3)
        block2_layer11=SeparableConv2D(64,(3,3), padding='valid',kernel_regularizer=regularizers.l2(0.00001),name='block2_sep_conv1')(block1_layer3)
        block2_layer1=Activation('relu',name='block2_relu1')(block2_layer11)
        block2_layer1=BatchNormalization(name='block2_batch1')(block2_layer1)
        block2_layer1=Dropout(0.2,name='block2_drop1')(block2_layer1)
        block2_layer2=SeparableConv2D(64,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block2_sep_conv2')(block2_layer1)
        block2_layer2=Activation('relu',name='block2_relu2')(block2_layer2)
        block2_layer2=BatchNormalization(name='block2_batch2')(block2_layer2)
        block2_layer2=Dropout(0.2,name='block2_drop2')(block2_layer2)
        block2_layer3=SeparableConv2D(64,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block2_sep_conv3')(block2_layer2)
        block2_add1=Add(name='block2_add')([block2_layer3,block2_layer11])
        block2_layer3=Activation('relu',name='block2_relu')(block2_add1)
        block2_layer3=BatchNormalization(name='block2_batch3')(block2_layer3)
        block2_layer3=ZeroPadding2D((3,3),name='block2_zeroo')(block2_layer3)
        block2_layer3=AveragePooling2D(name='block2_avg')(block2_layer3)
        block2_layer3=Dropout(0.2,name='block2_drop3')(block2_layer3)
        block3_layer11=SeparableConv2D(96,(3,3), padding='valid',kernel_regularizer=regularizers.l2(0.00001),name='block3_sep_conv1')(block2_layer3)
        block3_layer1=Activation('relu',name='block3_relu1')(block3_layer11)
        block3_layer1=BatchNormalization(name='block3_batch1')(block3_layer1)
        block3_layer1=Dropout(0.25,name='block3_drop1')(block3_layer1)
        block3_layer2=SeparableConv2D(96,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block3_sep_conv2')(block3_layer1)
        block3_layer2=Activation('relu',name='block3_relu2')(block3_layer2)
        block3_layer2=BatchNormalization(name='block3_batch2')(block3_layer2)
        block3_layer2=Dropout(0.25,name='block3_drop2')(block3_layer2)
        block3_layer3=SeparableConv2D(96,(3,3), padding='same',kernel_regularizer=regularizers.l2(0.00001),name='block3_sep_conv3')(block3_layer2)
        block3_add1=Add(name='block3_add')([block3_layer3,block3_layer11])
        block3_layer3=Activation('relu',name='block3_relu')(block3_add1)
        block3_layer3=BatchNormalization(name='block3_batch3')(block3_layer3)
        block3_layer3=ZeroPadding2D((3,3),name='block3_zeroo')(block3_layer3)
        block3_layer3=AveragePooling2D(name='block3_avg')(block3_layer3)
        block3_layer3=Dropout(0.25,name='block3_drop3')(block3_layer3)
        Global_Pooling=GlobalMaxPooling2D(name='Global_Pooling')(block3_layer3)
        Drop_Global_Pooling=Dropout(0.25,name='dense_drop1')(Global_Pooling)
        Dens1=Dense(96,activation='relu',name='dense1')(Drop_Global_Pooling)
        Dens1=BatchNormalization(name='dense_batch')(Dens1)
        Drop_Dens1=Dropout(0.25,name='dense_drop2')(Dens1)

        Softmax_=Dense(7,activation='softmax',name='softmax')(Drop_Dens1)
        mymodel=Model(myinput, Softmax_)
        mymodel.summary()
        mymodel.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001) , metrics=['accuracy'])

        return mymodel


In [8]:
model = generate_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 block1_sep_conv1 (Separabl  (None, 46, 46, 32)           73        ['input_1[0][0]']             
 eConv2D)                                                                                         
                                                                                                  
 block1_relu1 (Activation)   (None, 46, 46, 32)           0         ['block1_sep_conv1[0][0]']    
                                                                                                  
 block1_batch1 (BatchNormal  (None, 46, 46, 32)           128       ['block1_relu1[0][0]']    

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 block1_sep_conv1 (Separabl  (None, 46, 46, 32)           73        ['input_1[0][0]']             
 eConv2D)                                                                                         
                                                                                                  
 block1_relu1 (Activation)   (None, 46, 46, 32)           0         ['block1_sep_conv1[0][0]']    
                                                                                                  
 block1_batch1 (BatchNormal  (None, 46, 46, 32)           128       ['block1_relu1[0][0]']    

In [ ]:
if __name__=="__main__":

    x_train, y_train, x_valid, y_valid, x_test, y_test =  generate_dataset()

    x_train = x_train.reshape((-1,48,48,1)).astype(np.float32)
    x_valid = x_valid.reshape((-1,48,48,1)).astype(np.float32)
    x_test = x_test.reshape((-1,48,48,1)).astype(np.float32)
    filepath="/content/drive/My Drive/weights.bestvggtest-a.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    x_train_std = x_train/255.
    x_valid_std = x_valid/255.
    x_test_std = x_test/255.
    aug = keras.preprocessing.image.ImageDataGenerator(rotation_range=0.25,width_shift_range=0.1,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest")
    with tf.device("/gpu:0"):
         history = model.fit_generator(aug.flow(np.array(x_train_std),np.array(y_train),batch_size=64),epochs=100,callbacks=callbacks_list,validation_data=(np.array(x_valid_std),np.array(y_valid)),steps_per_epoch=len(np.array(x_train_std))/64)
         loss_test, acc_test=model.evaluate(x_test_std,y_test)
         print(loss_test, acc_test)

Epoch 1/100


<ipython-input-10-49bdfabf3360>:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(aug.flow(np.array(x_train_std),np.array(y_train),batch_size=64),epochs=100,callbacks=callbacks_list,validation_data=(np.array(x_valid_std),np.array(y_valid)),steps_per_epoch=len(np.array(x_train_std))/64)


448/448 [============================>.] - ETA: 0s - loss: 2.0206 - accuracy: 0.2071
Epoch 1: val_loss improved from inf to 1.81860, saving model to /content/drive/My Drive/weights.bestvggtest-a.h5
448/448 [==============================] - 48s 51ms/step - loss: 2.0202 - accuracy: 0.2070 - val_loss: 1.8186 - val_accuracy: 0.2494


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
448/448 [============================>.] - ETA: 0s - loss: 1.8332 - accuracy: 0.2403
Epoch 2: val_loss improved from 1.81860 to 1.77188, saving model to /content/drive/My Drive/weights.bestvggtest-a.h5
448/448 [==============================] - 23s 50ms/step - loss: 1.8331 - accuracy: 0.2406 - val_loss: 1.7719 - val_accuracy: 0.2636
Epoch 3/100
448/448 [============================>.] - ETA: 0s - loss: 1.7945 - accuracy: 0.2568
Epoch 3: val_loss did not improve from 1.77188
448/448 [==============================] - 32s 72ms/step - loss: 1.7944 - accuracy: 0.2569 - val_loss: 2.0970 - val_accuracy: 0.1803
Epoch 4/100
449/448 [==============================] - ETA: 0s - loss: 1.7682 - accuracy: 0.2741
Epoch 4: val_loss improved from 1.77188 to 1.74480, saving model to /content/drive/My Drive/weights.bestvggtest-a.h5
448/448 [==============================] - 22s 48ms/step - loss: 1.7682 - accuracy: 0.2741 - val_loss: 1.7448 - val_accuracy: 0.2641
Epoch 5/100
449/448 [========

In [ ]:
    import matplotlib as plt
    print(loss_test, acc_test)
    h=history.history
    h.keys()
    v_loss=h['val_loss']
    v_acc=h['val_accuracy']
    loss=h['loss']
    acc=h['accuracy']
    import matplotlib.pyplot as plt
    plt.plot(v_loss)
    plt.plot(v_acc)
    plt.plot(loss)
    plt.plot(acc)
    plt.legend(['val_loss','val_acc','loss','acc'])
    plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
    x_train, y_train, x_valid, y_valid, x_test, y_test =  generate_dataset()

    x_train = x_train.reshape((-1,48,48,1)).astype(np.float32)
    x_valid = x_valid.reshape((-1,48,48,1)).astype(np.float32)
    x_test = x_test.reshape((-1,48,48,1)).astype(np.float32)
    x_train_std = x_train/255.
    x_valid_std = x_valid/255.
    x_test_std = x_test/255.
    loss_test, acc_test=model.evaluate(x_test_std,y_test)
    print(loss_test, acc_test)

In [ ]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")


In [ ]:
model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')


In [ ]:
import cv2
import urllib
xml_url = "https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml"
urllib.request.urlretrieve(xml_url, "haarcascade_frontalface_default.xml")
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import model_from_json

face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

cap = cv2.VideoCapture(0)

emotion_labels = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        continue
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_haar_cascade.detectMultiScale(gray_image)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray_image[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        image_pixels = img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels, axis=0)
        image_pixels /= 255.0

        predictions = loaded_model.predict(image_pixels)
        max_index = np.argmax(predictions[0])
        emotion = emotion_labels[max_index]
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import model_from_json

face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

cap = cv2.VideoCapture(1)

emotion_labels = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        continue

    height, width, channel = frame.shape
    sub_img = frame[0:int(height/6), 0:int(width)]
    black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 0)

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    FONT_SCALE = 0.8
    FONT_THICKNESS = 2
    label_color = (10, 10, 255)
    label = "Emotion Detection"
    label_dimension = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)[0]
    textX = int((res.shape[1] - label_dimension[0]) / 2)
    textY = int((res.shape[0] + label_dimension[1]) / 2)
    cv2.putText(res, label, (textX, textY), FONT, FONT_SCALE, (0, 0, 0), FONT_THICKNESS)

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)

    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5]
            roi_gray = cv2.resize(roi_gray, (48, 48))
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)
            image_pixels /= 255.0

            predictions = loaded_model.predict(image_pixels)
            max_index = np.argmax(predictions[0])
            emotion = emotion_labels[max_index]

            cv2.putText(res, "Sentiment: {}".format(emotion), (0, textY + 22 + 5), FONT, 0.7, label_color, 2)
            label_confidence = 'Confidence: {}'.format(str(np.round(np.max(predictions[0]) * 100, 1)) + "%")
            confidence_text_dimension = cv2.getTextSize(label_confidence, FONT, FONT_SCALE, FONT_THICKNESS)[0]
            confidence_x_axis = int(res.shape[1] - confidence_text_dimension[0])
            cv2.putText(res, label_confidence, (confidence_x_axis, textY + 22 + 5), FONT, 0.7, label_color, 2)
    except:
        pass

    frame[0:int(height/6), 0:int(width)] = res
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows
